In [1]:
import os
from data.source.pg_experiment import get_pg_experiment_dataframe
import polars as pl

from models.SimplifiedLightweightCNN import SimplifiedLightweightCNN
%load_ext autoreload
%autoreload 1
%aimport models.SimplifiedLightweightCNN
from models.SimpleCNN_v2 import train, evaluate
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [18]:
df_pron, df_tone = get_pg_experiment_dataframe(".ogg")

dataframe = df_pron.with_columns(word_id = pl.struct("word_id").rank("dense"))
# dataframe = dataframe.filter(pl.col("mother") == "polish")
dataframe = dataframe.filter(pl.col("stage") == 1)
# df_stageI_polish = df_stageI_polish.filter(pl.col("word_id") == 1)

N_WORDS = dataframe.select(pl.col("word_id").n_unique()).to_numpy()[0][0]
print(f"Number of unique words: {N_WORDS}")
print(f"Number of samples: {dataframe.shape[0]}")

<sys>:0: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
<sys>:0: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


get_pg_experiment_dataset(): WARNING, Dropped 2 rows with missing files
get_pg_experiment_dataset(): WARNING, Dropped 2 rows with missing files
Number of unique words: 12
Number of samples: 6016


In [19]:
import polars as pl
import numpy as np
from typing import Tuple

def stratified_split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [22]:
from typing import Callable

from polars import DataFrame
from dataset import Cast, TorchDataset
from develop import reload_function, reload_module
import pytorch_dataloader
reload_module(pytorch_dataloader)
from pytorch_dataloader import ReshapeCollate, build_collate_fn, PaddingCollate, DefaultCollate
from functools import partial

from transformation import Channels, RMSEnergy, TorchVadLogMelSpec, TorchVadMFCC, ZeroCrossingRate

reload_function(TorchVadMFCC)

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = stratified_split(dataframe, label_col="value", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)

to_dataset: Callable[[DataFrame], TorchDataset] = lambda dataframe: TorchDataset(
    Cast(dataframe.get_column("rec_path"), Channels("stack","multiply")(
            TorchVadMFCC(delta=0),
            TorchVadMFCC(delta=1),
            TorchVadMFCC(delta=2),
        )),
    Cast(dataframe.get_column("rec_path"), Channels("cat","multiply")(
            ZeroCrossingRate(),
            RMSEnergy(),
        )),
    Cast(dataframe.get_column("word_id"), lambda x: torch.tensor(x-1, dtype=torch.long)),
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=2),
    PaddingCollate(mode="SET_MAX_LEN", max_len=160, pad_dim=1),
    DefaultCollate(),
    DefaultCollate(),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [23]:
from pytorch_dataloader import MemoryLoadedDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=4)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)

for x in next(iter(train_loader)):
    print(x.shape)

torch.Size([16, 3, 40, 80])
torch.Size([16, 2, 160])
torch.Size([16])
torch.Size([16])


In [24]:
train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")

/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/1536/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/1536/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/1536/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/765/a5.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/765/a5.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/765/a5.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Ma

In [54]:
from models.FusionCNN import ContextFusionCNN
reload_function(ContextFusionCNN)
model = ContextFusionCNN(3,2, num_words=N_WORDS)

In [55]:
# Model variables definition.
pth = "SimplifiedLightweightCNN.pth"
lr = 1e-4  # Reduce from 1e-3
epochs = 200
model = model.to(device)
reload_function(train)
reload_function(evaluate)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)  # Add L2 regularization
# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)
criterion = nn.BCELoss()

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the run
    name="Hypothesis-5+2 ContextFusionCNN(16) 2L-NN",
    config={
        "Name": 'SimplifiedLightweightCNN',
        "learning_rate": lr,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "SimplifiedLightweightCNN",
        "architecture_details": str(model),
        "dataset": "Stage-I",
        "train_val_test(%)": f'{TRAIN_SPLIT}-{VAL_SPLIT}-{TEST_SPLIT}',
        "epochs": epochs,
    },
)

# Training loop
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    # Update learning rate
    scheduler.step(val_loss)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(
        f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("SimplifiedLightweightCNN-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.6904, Train Acc: 0.5220, Val Loss: 0.6843, Val Acc: 0.5960
Epoch 2, Train Loss: 0.6832, Train Acc: 0.5409, Val Loss: 0.6781, Val Acc: 0.5952
Epoch 3, Train Loss: 0.6775, Train Acc: 0.5719, Val Loss: 0.6716, Val Acc: 0.6035
Epoch 4, Train Loss: 0.6746, Train Acc: 0.5689, Val Loss: 0.6671, Val Acc: 0.6035
Epoch 5, Train Loss: 0.6683, Train Acc: 0.5957, Val Loss: 0.6607, Val Acc: 0.6068
Epoch 6, Train Loss: 0.6649, Train Acc: 0.5960, Val Loss: 0.6547, Val Acc: 0.6126
Epoch 7, Train Loss: 0.6589, Train Acc: 0.6007, Val Loss: 0.6483, Val Acc: 0.6193
Epoch 8, Train Loss: 0.6559, Train Acc: 0.5874, Val Loss: 0.6431, Val Acc: 0.6135
Epoch 9, Train Loss: 0.6494, Train Acc: 0.6074, Val Loss: 0.6356, Val Acc: 0.6243
Epoch 10, Train Loss: 0.6399, Train Acc: 0.6129, Val Loss: 0.6307, Val Acc: 0.6342
Epoch 11, Train Loss: 0.6321, Train Acc: 0.6198, Val Loss: 0.6236, Val Acc: 0.6442
Epoch 12, Train Loss: 0.6317, Train Acc: 0.6262, Val Loss: 0.6208, Val Acc: 0.6492
Epoch 13, Tra

KeyboardInterrupt: 